In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Ridge, Lasso
import statsmodels as sm
from statsmodels.api import OLS

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

import sys
sys.path.insert(0,'../src')
from data import build_features
import config

**Note**: Statsmodels library generally preferable to sklearn as gives more detailed output out-of-the-box.

## Load data

In [ ]:
combined_df = build_features.process_data()

## Set parameters

In [3]:
predicted_variable = 'variable_X'

In [ ]:
model_type = 'Lasso' #'Ridge' # 'OLS'

## Prepare data

In [ ]:
input_df = combined_df.drop(config.prediction_outputs, axis=1)
target_df = combined_df[predicted_variable]

In [ ]:
# Split data to create train and test data
train_X, test_X = train_test_split(input_df, random_state=42)
train_Y, test_Y = train_test_split(target_df, random_state=42)

## Parameter selection

In [4]:
# TODO: add step-wise feature selection and/or alternatives

## Fit model

In [ ]:
model = OLS(train_Y, train_X)

In [ ]:
if model_type == 'OLS':
    results = model.fit()
elif model_type == 'Lasso':
    results = model.fit_regularized(method='sqrt_lasso', L1_wt = 1) #  refit=True, zero_tol = 0.01b
elif model_type == 'Ridge':
    results = model.fit_regularized(method='sqrt_lasso', L1_wt = 0)

In [ ]:
if model_type == 'OLS':
    coeffs = results.params
    tvalues = results.tvalues
    num_features = len(list(train_X))
    F_test = results.f_test(np.identity(num_features))

    
else:
    coeffs = results.params
    # vals = results.fittedvalues


In [ ]:
if model_type == 'OLS':
    results.summary() # Print full summary information
    # results.summary().tables[1] # Print coefficients and t-values
    
    # NOTE: statsmodels summary() function only implemented for OLS - not for regularised models

## Assess fit

In [ ]:
# Generate predictions
predictions = model.predict(results.params, exog=test_X)

In [ ]:
print(f"MSE: {mean_squared_error(test_Y.values, predictions)}")
print(f"RMSE: {mean_squared_error(test_Y.values, predictions, squared=False)}")
print(f"R^2: {r2_score(test_Y.values, predictions)}")

In [ ]:
predictions_df = pd.DataFrame(test_Y)
predictions_df['predictions'] = predictions

In [ ]:
# Plot predicted vs true. 
# NOTE: scale 0 to 1 of x-axis for normalised. (comment out if fitting unnormalised)
plt.scatter(predictions_df['BPI_Impact_Change'], predictions_df['predictions'])
plt.xlim(0,1)
plt.ylim(0,1)
plt.xlabel("true")
plt.ylabel("predicted")
plt.show()

In [ ]:
# look at exact predictions and actual
predictions_df

## Interrogate model

Rank the coefficients in order.

In [79]:
# Make coefficients absolute
coeffs_abs = np.abs(coeffs)

In [80]:
# Get indices of sort in descending order
coef_sort_order = coeffs_abs.argsort()[::-1]

In [84]:
for index in coef_sort_order:
    print(f"{coeffs.index[index]}:\t\t\t\t {coeffs[index]}.")

Trazodone:				 -0.2609201077219451.
Age:				 0.23422858921924386.
Back Pain:				 0.2018076230118869.
Pred:				 0.14370233484778133.
Biologics:				 -0.12274251979957868.
Disability_received:				 0.12037916680348124.
Sympathicomimetics:				 0.11629015280678834.
Neuroleptics:				 0.10922355837360542.
BMI_v_low:				 0.10242728770318324.
Nociceptive:				 -0.09228681747545155.
Neuropathic Pain:				 -0.08927701027646712.
sex:				 0.08620495141840304.
Child/Adolescent pain:				 0.08301794577027376.
Disability_demanded:				 0.07240395787156964.
Opiates_weak:				 0.07128341313682388.
Mirtazapin:				 0.06965253213274387.
Other psych conds:				 -0.06324401186535417.
Disability_refused:				 0.06236877481864439.
Disturbed sleep:				 0.06095700181207002.
Airway comorbidities:				 -0.05839767886434518.
Z-Drugs:				 0.05722628485287026.
Confirmed auto-immune rheumatism:				 0.0570423154149589.
Alexithymia:				 0.05346768786481004.
Tricyclics ou quadriciclys:				 0.051831266310203315.
Dual:				 0.047419